In [1]:
# This notebook is divided into two main parts:

# Part 1: Model Validation
# In this section, the focus is on validating pre-trained models. 
# It involves assessing the performance of models that have already been trained, 
# using predefined parameters such as time index ranges for the validation dataset. 
# The goal is to evaluate the predictive accuracy of these models on test data.

# Part 2: Model Training
# This section is dedicated to training your own models. 
# It guides you through the process of setting up and configuring a new model, 
# including defining its parameters and architecture. 
# You will train the model using your dataset, allowing for experimentation 
# with different configurations to develop a model that best suits your data and prediction objectives.


In [2]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

import tensorflow as tf 
import tensorboard as tb 
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss,MAE,MAPE,RMSE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from pytorch_lightning.callbacks import ModelCheckpoint

In [3]:
import pandas as pd

data = pd.read_excel('./dataset/EMNN Decomposed Data.xlsx')
data["day"] = data["day"].astype(str)
data["WTI"] = data["WTI"].astype("float64")
data["month"] = data["month"].astype(str)
data["weekday"] = data["weekday"].astype(str)

In [4]:
# create dataloaders for model
batch_size = 256  # set this between 32 to 128


# check if GPU is available
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# move dataloaders to device


## Part 1: Model Validation

In [25]:
# IMF 1

In [66]:
# Used for storing the trained model
pretrained_model_paths=['.\\TFT_saved_models\\best_model_epoch=19-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=18.ckpt', '.\\TFT_saved_models\\best_model_epoch=1-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=1-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=6-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=12.ckpt', '.\\TFT_saved_models\\best_model_epoch=7.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=22.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v13.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v13.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=13-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=18-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=16.ckpt', '.\\TFT_saved_models\\best_model_epoch=34.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v15.ckpt', '.\\TFT_saved_models\\best_model_epoch=1-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=14.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v16.ckpt', '.\\TFT_saved_models\\best_model_epoch=15.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v17.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v18.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v19.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v15.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v20.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v16.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v21.ckpt', '.\\TFT_saved_models\\best_model_epoch=1-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=39.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v22.ckpt', '.\\TFT_saved_models\\best_model_epoch=47.ckpt', '.\\TFT_saved_models\\best_model_epoch=1-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v23.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v18.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v24.ckpt', '.\\TFT_saved_models\\best_model_epoch=13-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v25.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v19.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v26.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v27.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v20.ckpt', '.\\TFT_saved_models\\best_model_epoch=1-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=15-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=22-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=10-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=12-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v29.ckpt', '.\\TFT_saved_models\\best_model_epoch=15-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v21.ckpt', '.\\TFT_saved_models\\best_model_epoch=21-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v30.ckpt', '.\\TFT_saved_models\\best_model_epoch=18-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=10-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v31.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v22.ckpt', '.\\TFT_saved_models\\best_model_epoch=18-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v32.ckpt', '.\\TFT_saved_models\\best_model_epoch=5.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v33.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v34.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v23.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v35.ckpt', '.\\TFT_saved_models\\best_model_epoch=11.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v36.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v37.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v24.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v25.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v38.ckpt', '.\\TFT_saved_models\\best_model_epoch=8.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v39.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v26.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v27.ckpt']

In [67]:
start_prediction_idx = 713
end_prediction_idx = 802
prediction_length = 1  # The step length for each prediction
max_encoder_length=30

# Initialize an empty list to store all predictions
all_predictions = []


In [68]:
# Used for creating the validation set
training = TimeSeriesDataSet(
    data[(data.time_idx >= 1) & (data.time_idx <= 712)],
    time_idx="time_idx",
    target="WTI_IMF1",
    min_encoder_length=max_encoder_length // 2, 
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=prediction_length,
    time_varying_known_categoricals=["month","weekday","day"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "WTI_IMF1",
        
    ],
    group_ids=['destination'],
    target_normalizer=GroupNormalizer(
        groups=['destination'], transformation="softplus"),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,

   
)

In [69]:
# Loop for making predictions
for i, model_path in enumerate(pretrained_model_paths):
    start_idx = start_prediction_idx + i * prediction_length
    end_idx = start_idx + prediction_length

    if end_idx > end_prediction_idx:
        end_idx = end_prediction_idx + 1  
    
    # Create a new validation dataset
    validation_data = data[data['time_idx'] < end_idx].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    # Load the pretrained model and make predictions
    best_tft = TemporalFusionTransformer.load_from_checkpoint(model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    current_prediction = raw_predictions[0][:, :, 3]  # Retrieve current prediction results
    current_prediction_numpy = current_prediction.numpy().flatten()
    all_predictions.extend(current_prediction_numpy.tolist())

IMF1_fore=all_predictions

In [ ]:
# IMF2

In [70]:
pretrained_model_paths=['.\\TFT_saved_models\\best_model_epoch=7-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=11-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=43.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=7-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=7-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=7-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=10-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=42.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v40.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=33.ckpt', '.\\TFT_saved_models\\best_model_epoch=10-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=44-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v41.ckpt', '.\\TFT_saved_models\\best_model_epoch=10-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=43-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=27-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=43-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=43-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=28-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=33-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=40-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=26-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v51.ckpt', '.\\TFT_saved_models\\best_model_epoch=7-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=27-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=26-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=36-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=45-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=32-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v13.ckpt', '.\\TFT_saved_models\\best_model_epoch=41.ckpt', '.\\TFT_saved_models\\best_model_epoch=27-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=28-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=37-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=22-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=43-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=28-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=34-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=25-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=37-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=5-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=45-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v52.ckpt', '.\\TFT_saved_models\\best_model_epoch=45-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=39-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=36-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=38.ckpt', '.\\TFT_saved_models\\best_model_epoch=24-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v53.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v13.ckpt', '.\\TFT_saved_models\\best_model_epoch=36-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=45-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=45-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=18-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v37.ckpt', '.\\TFT_saved_models\\best_model_epoch=40-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v38.ckpt', '.\\TFT_saved_models\\best_model_epoch=21-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=18-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=34-v15.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v39.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v21.ckpt', '.\\TFT_saved_models\\best_model_epoch=40-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=30-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v54.ckpt', '.\\TFT_saved_models\\best_model_epoch=36-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v55.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v56.ckpt', '.\\TFT_saved_models\\best_model_epoch=12-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=30-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=39-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=16-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=22-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=17.ckpt', '.\\TFT_saved_models\\best_model_epoch=30-v7.ckpt']

In [71]:
start_prediction_idx = 713
end_prediction_idx = 802
prediction_length = 1  # The step length for each prediction
max_encoder_length=30

# Initialize an empty list to store all predictions
all_predictions = []


In [72]:
# Used for creating the validation set
training = TimeSeriesDataSet(
    data[(data.time_idx >= 1) & (data.time_idx <= 712)],
    time_idx="time_idx",
    target="WTI_IMF2",
    min_encoder_length=max_encoder_length // 2, 
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=prediction_length,
    time_varying_known_categoricals=["month","weekday","day"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "WTI_IMF2",
        
    ],
    group_ids=['destination'],
    target_normalizer=GroupNormalizer(
        groups=['destination'], transformation="softplus"),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,

   
)

In [73]:
# Loop for making predictions
for i, model_path in enumerate(pretrained_model_paths):
    start_idx = start_prediction_idx + i * prediction_length
    end_idx = start_idx + prediction_length

    if end_idx > end_prediction_idx:
        end_idx = end_prediction_idx + 1  
    
    # Create a new validation dataset
    validation_data = data[data['time_idx'] < end_idx].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    # Load the pretrained model and make predictions
    best_tft = TemporalFusionTransformer.load_from_checkpoint(model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    current_prediction = raw_predictions[0][:, :, 3]  # Retrieve current prediction results
    current_prediction_numpy = current_prediction.numpy().flatten()
    all_predictions.extend(current_prediction_numpy.tolist())

IMF2_fore=all_predictions

In [ ]:
# Residual

In [78]:
pretrained_model_paths=['.\\TFT_saved_models\\best_model_epoch=38-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=22-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=38-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=11-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=38-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=38-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=16-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=38-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=38-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=22-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=38-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=36-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=36-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=15-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=11-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=36-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=22-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=11-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=22-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v62.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v62.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=22-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=24-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=11-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=15-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=24-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=22-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=24-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v62.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v62.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=15-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=15-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v67.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v62.ckpt', '.\\TFT_saved_models\\best_model_epoch=15-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v28.ckpt', '.\\TFT_saved_models\\best_model_epoch=15-v3.ckpt']

In [79]:
start_prediction_idx = 713
end_prediction_idx = 802
prediction_length = 1  # The step length for each prediction
max_encoder_length=30

# Initialize an empty list to store all predictions
all_predictions = []


In [80]:
# Used for creating the validation set
training = TimeSeriesDataSet(
    data[(data.time_idx >= 1) & (data.time_idx <= 712)],
    time_idx="time_idx",
    target="WTI_Res",
    min_encoder_length=max_encoder_length // 2, 
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=prediction_length,
    time_varying_known_categoricals=["month","weekday","day"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "WTI_Res",
        
    ],
    group_ids=['destination'],
    target_normalizer=GroupNormalizer(
        groups=['destination'], transformation="softplus"),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,

   
)

In [81]:
# Loop for making predictions
for i, model_path in enumerate(pretrained_model_paths):
    start_idx = start_prediction_idx + i * prediction_length
    end_idx = start_idx + prediction_length

    if end_idx > end_prediction_idx:
        end_idx = end_prediction_idx + 1  
    
    # Create a new validation dataset
    validation_data = data[data['time_idx'] < end_idx].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    # Load the pretrained model and make predictions
    best_tft = TemporalFusionTransformer.load_from_checkpoint(model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    current_prediction = raw_predictions[0][:, :, 3]  # Retrieve current prediction results
    current_prediction_numpy = current_prediction.numpy().flatten()
    all_predictions.extend(current_prediction_numpy.tolist())

Res_fore=all_predictions

In [83]:
true_values = [82.83, 84.36, 85.89, 83.7, 82.87, 87.67, 86.65, 86.66, 88.35, 89.35, 89.12, 85.49, 84.58, 86.07, 83.8, 86.04, 83.03, 81.64, 81.05, 83.04, 81.19, 81.54, 77.96, 75.85, 76.34, 77.6, 78.86, 78.9, 77.15, 73.5, 76.47, 78.1, 78.35, 76.8, 75.815, 74.83, 74.46, 76.09, 77.56, 75.66, 73.7, 72.73, 71.95, 68.98, 69.0, 70.87, 70.95, 68.27, 69.09, 71.21, 71.05, 72.16, 73.23, 73.87, 73.59, 73.29, 75.84, 74.31, 72.02, 71.89, 70.62, 72.97, 72.38, 74.0, 71.06, 72.43, 71.57, 72.15, 72.94, 72.785, 72.63, 72.79, 74.32, 73.69, 75.26, 74.72, 75.48, 77.91, 78.45, 77.25, 78.3, 76.28, 74.36, 72.72, 73.21, 73.83, 74.26, 76.67, 77.26, 77.34]

In [84]:
import numpy as np

# Combine IMF1, IMF2, and residual components to get the final forecast
final = [sum(x) for x in zip(IMF1_fore, IMF2_fore, Res_fore)]
# Adjust the elements in the 'final' list to balance the previous increments of 10 each to IMF1 and IMF2, 
# effectively removing the added 20 units per element.
final = [x - 20 for x in final]

# Calculate Mean Absolute Error (MAE), Root Mean Square Error (RMSE), and Mean Absolute Percentage Error (MAPE)
mae = np.mean(np.abs(np.array(true_values) - np.array(final)))
rmse = np.sqrt(np.mean(np.square(np.array(true_values) - np.array(final))))
mape = np.mean(np.abs(np.array(true_values) - np.array(final)) / np.array(true_values))

# Print the evaluation metrics
print("MAE:", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

MAE: 0.7718856141832139
RMSE: 1.2555244300722526
MAPE: 0.009821374345646963


## Part 2: Model Training

In [ ]:
# Before training the Temporal Fusion Transformer (TFT) model, 
# it is beneficial to employ the Tree-structured Parzen Estimator (TPE) method for hyperparameter optimization.
# This step aids in selecting the optimal hyperparameters for the TFT model. 
# However, the TPE method can be computationally intensive and involves randomness, 
# hence it is optional. We have already incorporated the optimal hyperparameters
# obtained from running this method into the corresponding TFT model.
# If you wish to perform this optimization and discover your own optimal hyperparameters,
# you may run the following code. Note that this process may significantly impact computational resources.

In [ ]:
# max_prediction_length = 1
# max_encoder_length = 30

# training = TimeSeriesDataSet(
#     data[(data.time_idx >= 1) & (data.time_idx <= 712)],
#     time_idx="time_idx",
#     target="WTI_IMF1",
#     min_encoder_length=max_encoder_length // 2, 
#     max_encoder_length=max_encoder_length,
#     min_prediction_length=1,
#     max_prediction_length=max_prediction_length,
#     time_varying_known_categoricals=["month","weekday","day"],
#     time_varying_known_reals=["time_idx"],
#     time_varying_unknown_categoricals=[],
#     time_varying_unknown_reals=[
#         "WTI_IMF1",
        
#     ],
#     group_ids=['destination'],
#     target_normalizer=GroupNormalizer(
#         groups=['destination'], transformation="softplus"),
#     add_relative_time_idx=True,
#     add_target_scales=True,
#     add_encoder_length=True,
#     allow_missing_timesteps=True,

   
# )
# validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)


# # create dataloaders for model
# batch_size = 256  # set this between 32 to 128


# # check if GPU is available
# if torch.cuda.is_available():
#     device = "cuda"
# else:
#     device = "cpu"

# # move dataloaders to device


# train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
# val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size*10 , num_workers=0)

In [ ]:
# import pickle
# from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters



# # create study
# study = optimize_hyperparameters(
#     train_dataloader,
#     val_dataloader,
#     model_path="optuna_test",
#     n_trials=50,
#     max_epochs=50,
#     gradient_clip_val_range=(0.01, 1.0),
#     hidden_size_range=(8, 128),
#     hidden_continuous_size_range=(8, 128),
#     attention_head_size_range=(1, 4),
#     learning_rate_range=(0.001, 0.1),
#     dropout_range=(0.1, 0.3),
#     trainer_kwargs=dict(limit_train_batches=30),
#     reduce_on_plateau_patience=4,
#     use_learning_rate_finder=False
# )


# # save study results - also we can resume tuning at a later point in time
# with open("test_study.pkl", "wb") as fout:
#     pickle.dump(study, fout)

# # show best hyperparameters
# print(study.best_trial.params)


In [85]:
# IMF 1

# Define the time index range for the validation dataset
start_prediction_idx = 713 # Starting index for prediction
end_prediction_idx = 802 # Ending index for prediction


# Set the prediction length to 1, indicating that the model predicts 1 steps ahead
prediction_length = 1

# Note: The test set comprises 90 time points. The prediction_length of 1 means that 
# the models trained in this session are designed to forecast 1 steps ahead. The 
# start and end indices (713 to 802) specify the time points covered by the 
# model's predictions. These indices also determine the number of models to be trained, 
# calculated as: (end_prediction_idx - start_prediction_idx + 1) / prediction_length.
# For start_prediction_idx = 713 and end_prediction_idx = 802 with prediction_length = 1,
# the total number of models trained in the loop is 90.

max_encoder_length = 30 

# Initialize an empty list to store all prediction values
all_predictions = []

# Initialize an empty list to store the best models for each prediction
all_best_models = []

In [86]:
# Caution: The models are resource-intensive, requiring a high-performance GPU. Training 
# too many models in a single loop may lead to exhaustion of computer resources, causing 
# the program to crash. In such cases, you may need to restart the program or adjust the 
# start and end indices to reduce the number of models trained in the loop.

In [ ]:
# Iterate over prediction indices to generate training and validation sets dynamically
for prediction_idx in range(start_prediction_idx, end_prediction_idx + 1, prediction_length):
    # Define the training dataset, filtering data up to the current prediction index
    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= prediction_idx - 1], 
        time_idx="time_idx",
        target="WTI_IMF1",
        min_encoder_length=max_encoder_length // 2, 
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=prediction_length,
        time_varying_known_categoricals=["month","weekday","day"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=[
            "WTI_IMF1",

        ],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(
            groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )
    
    # Copy validation data set based on prediction length and current index
    validation_data = data[lambda x: x.time_idx <= prediction_idx - 1 + prediction_length].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    # Create data loaders for training and validation
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)
    
    
    # Setup model checkpointing
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        mode="min",
        save_last=True,
        save_top_k=1,  
        filename="best_model_{epoch}", 
        dirpath="saved_models"
    )
    
    # Setup early stopping to prevent overfitting
    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    
    # Initialize logging for learning rate and training process
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard
    
    # Initialize trainer with specified parameters and callbacks
    trainer = pl.Trainer(
        max_epochs=50,
        gpus=1,
        enable_model_summary=True,
        gradient_clip_val=0.1259662302414041,
        limit_train_batches=30,  # coment in for training, running valiation every 30 batches
        # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
        callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
        logger=logger,
    )

    # Initialize the TFT model with specified hyperparameters
    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.006878237268828405,
        hidden_size=18,
        attention_head_size=3,
        dropout=0.2610076941913576,
        hidden_continuous_size=9,
        output_size=7,  # 7 quantiles by default
        loss=QuantileLoss(),
        log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        reduce_on_plateau_patience=4,
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


    # Train the model
    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )
    
    # Log and store the path of the best performing model
    best_model_path = trainer.checkpoint_callback.best_model_path
    current_model=best_model_path
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
    
    # Perform predictions using the best model
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    all_best_models.append(current_model)
    current_prediction = raw_predictions[0][:, :, 3]  
    all_predictions.append(current_prediction)


In [ ]:
all_predictions 

In [ ]:
all_best_models

In [ ]:
# IMF 2

In [5]:
start_prediction_idx = 713
end_prediction_idx = 802 

prediction_length = 1
max_encoder_length = 30 

all_predictions = []
all_best_models = []

In [ ]:

for prediction_idx in range(start_prediction_idx, end_prediction_idx + 1, prediction_length):
    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= prediction_idx - 1], 
        time_idx="time_idx",
        target="WTI_IMF2",
        min_encoder_length=max_encoder_length // 2, 
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=prediction_length,
        time_varying_known_categoricals=["month","weekday","day"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=[
            "WTI_IMF2",

        ],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(
            groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )

    validation_data = data[lambda x: x.time_idx <= prediction_idx - 1 + prediction_length].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)
    
    
    # configure network and trainer
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        mode="min",
        save_last=True,
        save_top_k=1,  
        filename="best_model_{epoch}",  
        dirpath="saved_models"
    )

    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=50,
        gpus=1,
        enable_model_summary=True,
        gradient_clip_val=0.7808891799998493,
        limit_train_batches=30,  # coment in for training, running valiation every 30 batches
        # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
        callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
        logger=logger,
    )


    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.017684809018645165,
        hidden_size=121,
        attention_head_size=1,
        dropout=0.2930918827522154,
        hidden_continuous_size=106,
        output_size=7,  # 7 quantiles by default
        loss=QuantileLoss(),
        log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        reduce_on_plateau_patience=4,
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


    
    
    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    best_model_path = trainer.checkpoint_callback.best_model_path
    current_model=best_model_path
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    all_best_models.append(current_model)
    current_prediction = raw_predictions[0][:, :, 3]  
    all_predictions.append(current_prediction)


In [ ]:
all_predictions 

In [ ]:
all_best_models

In [7]:
# Residual

In [8]:
start_prediction_idx = 713
end_prediction_idx = 802 

prediction_length = 1
max_encoder_length = 30 

all_predictions = []
all_best_models = []

In [ ]:
for prediction_idx in range(start_prediction_idx, end_prediction_idx + 1, prediction_length):
    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= prediction_idx - 1],  
        time_idx="time_idx",
        target="WTI_Res",
        min_encoder_length=max_encoder_length // 2, 
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=prediction_length,
        time_varying_known_categoricals=["month","weekday","day"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=[
            "WTI_Res",

        ],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(
            groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )

    validation_data = data[lambda x: x.time_idx <= prediction_idx - 1 + prediction_length].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)
    
    
    # configure network and trainer
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        mode="min",
        save_last=True,
        save_top_k=1,  
        filename="best_model_{epoch}",  
        dirpath="saved_models"
    )

    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=50,
        gpus=1,
        enable_model_summary=True,
        gradient_clip_val=0.07736307854495153,
        limit_train_batches=30,  # coment in for training, running valiation every 30 batches
        # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
        callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
        logger=logger,
    )


    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.006180254073038927,
        hidden_size=20,
        attention_head_size=2,
        dropout=0.266097695477857,
        hidden_continuous_size=10,
        output_size=7,  # 7 quantiles by default
        loss=QuantileLoss(),
        log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        reduce_on_plateau_patience=4,
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


    
    
    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    best_model_path = trainer.checkpoint_callback.best_model_path
    current_model=best_model_path
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    all_best_models.append(current_model)
    current_prediction = raw_predictions[0][:, :, 3]  
    all_predictions.append(current_prediction)


In [ ]:
all_predictions 

In [ ]:
all_best_models